## Решение задачи классификации текстов

Возьмем в качестве примера данные новостей `fetch_20newsgroups` из sklearn и рассмотрим категории "атеизм" и "космос".

In [71]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

In [2]:
data = fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])

Массив с текстами будет находиться в поле `data.data`, номер класса $-$ в поле `data.target`. В этой задаче для преобразования текстов будем использовать метод Tf-Idf.

**Задание 1.** Преобразуйте тексты с помощью `TfIdfVectorizer`.

In [107]:
TfidfVectorizer?

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
tfdf_vec = TfidfVectorizer()
news_vec_tfdf = tfdf_vec.fit_transform(data.data)

In [10]:
news_vec_tfdf.shape

(1786, 28382)

In [15]:
y = data.target
y.shape

(1786,)

Реализация SVM находится в классе `sklearn.svm.SVC`. Веса каждого признака у обученного классификатора хранятся в поле `coef_`.

**Задание 2.** С помощью функции `sklearn.model_selection.GridSearchCV` выберите лучший параметр `C` для SVM с линейным ядром (`kernel='linear'`) из множества $[10^{-5}, 10^{-4}, 10^{-3}, ... , 10^{3}, 10^{4}, 10^{5}]$. Кросс-валидацию выполняйте по 5 блокам с параметром `random_state=241` и
для `SVC`, и для `KFold`. Качество оценивайте с помощью метрики `accuracy`.

In [96]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(news_vec_tfdf, y,
                 test_size=0.3, random_state=42)

In [88]:
cv = KFold(n_splits=5, shuffle=True, random_state=241)
from sklearn.model_selection import GridSearchCV

C = [10**(N+1) for N in range(-5,5)]
param_grid = {'C': C}

best_SVC = GridSearchCV(SVC(kernel = 'linear', random_state=241), 
                      param_grid=param_grid, 
                      scoring='accuracy',
                      n_jobs=-1,
                      cv = cv)


# Поиск наилучшего классификатора
res_best_SVC = best_SVC.fit(news_vec_tfdf, y)

In [89]:
param = res_best_SVC.best_estimator_.get_params()
param

{'C': 1,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'linear',
 'max_iter': -1,
 'probability': False,
 'random_state': 241,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [90]:
res_best_SVC.best_score_

0.9932804406678872

**Задание 3.** Обучите SVM на всей выборке с лучшим параметром `C`. Найдите 10 слов с наибольшим по модулю весом.

In [103]:
best_model = SVC()
best_model.set_params(**param)
best_model.fit(news_vec_tfdf, y)

SVC(C=1, kernel='linear', random_state=241)

In [143]:
def get_best_words(tfdf_vec, model, n = 10):
    coefs = model.coef_.toarray().ravel()
    ind = coefs.argsort()[-n:][::-1].ravel()
    return np.array(tfdf_vec.get_feature_names())[ind]

In [144]:
coefs = best_model.coef_.toarray()

In [145]:
result_10_words = get_best_words(tfdf_vec, best_model)
result_10_words

array(['space', 'moon', 'sky', 'sci', 'nasa', 'dc', 'launch', 'gamma',
       'ca', 'ether'], dtype='<U80')

In [146]:
import pandas as pd
pd.DataFrame(result_10_words)

,0
0,space
1,moon
2,sky
3,sci
4,nasa
5,dc
6,launch
7,gamma
8,ca
9,ether


## Дополнительное задание

Решить задачи `task1` и `task2` с помощью SVM из sklearn. В задаче `task2` провести сравнение на графике различных функций ядра.